In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

Note: you may need to restart the kernel to use updated packages.


In [21]:
import getpass
import os
os.environ['USER_AGENT']='ProfessorAgent'

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [49]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Vald
from langchain_text_splitters import CharacterTextSplitter

raw_documents = TextLoader("../data/data.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [50]:
docs = text_splitter.split_documents(documents)

In [51]:
len(docs[0].page_content)

324993

In [52]:
print(docs[0].page_content[5000:8000])

tion virtualization because I will come back to this once I explained Holograms and volumetric media delivery then I will also Explain how virtual Network functions can be of interest so if we consider a home environments then there are multiple devices and in your home and typically there is a home Gateway. And on the home K3, we have different functionality. There is the HTTP server. There is an entity router firewall also quality of experience monitoring software for instance. if we have TV streams or video streams that we display on screens inside your home then we also monitor the quality of experience inside the home Gateway. And then typical Auto device is a setup box. so stb. Which contains? for a digital TV storage so that you can record movies and TV programs. Middleware that allows you to interact through remote control. And to start recording stop recording play a certain TV program. and so on for unless we need to middleware on the setup box. And also we have a streaming c

In [53]:
embeddings = OpenAIEmbeddings()
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

In [54]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [55]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [61]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo", max_tokens=2048)
db = DocArrayInMemorySearch.from_documents(
    all_splits,
    embeddings
)
retriever = db.as_retriever()
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [62]:
from langchain.agents import tool

@tool
def find_seminar_quote(text: str) -> str:
    """ Receives a question from the user about a seminar and returns a description of it from the transcript.
    """

    query =  f"""Please gather more information about '''{text}''' and summarize the output"""
    response = qa_stuff.run(query)
    
    return response


In [63]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_community.tools.tavily_search import TavilySearchResults

tools=[find_seminar_quote]
tools = tools # +  [TavilySearchResults(max_results=1)]# + load_tools(["wikipedia"], llm=llm)
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [40]:
agent("Based on what you know what is the most efficient techinique to prompt an llm?")



> Entering new AgentExecutor chain...
Thought: I should use the `find_seminar_quote` tool to gather information on the most efficient technique to prompt an LLM.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "most efficient technique to prompt an LLM"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: I don't have specific information on the most efficient technique to prompt a Large Language Model (LLM). However, based on the context provided, it seems that formatting or prompting engineering is a powerful tool to maximize the capabilities of autocomplete systems like LLMs. One approach mentioned is using demonstrations or chaining thoughts to provide examples for the LLM to learn from. Another technique discussed is editing within a sentence by reshuffling and adding special words to train the model to predict in the middle of a sentence. These methods can enhance the performance of LLMs without compromising their overall capabilities. 

{'input': 'Based on what you know what is the most efficient techinique to prompt an llm?',
 'output': 'The most efficient technique to prompt a Large Language Model (LLM) involves leveraging formatting techniques and providing ample demonstrations to help the LLM understand patterns and examples effectively. This includes using demonstrations or chaining thoughts, editing within a sentence, and ensuring a sufficient number of demonstrations (around 50) to enhance the performance of LLMs without compromising their overall capabilities.'}

In [44]:
agent("How much data will be necessary to send an hologram through our networks?")



> Entering new AgentExecutor chain...
Thought: I should use the find_seminar_quote tool to see if there is any information about sending holograms through networks in the seminar transcript.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "sending holograms through networks"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: The technology discussed involves sending holograms through networks using techniques like point clouds and adaptive streaming. Point clouds represent objects by considering all possible points in three-dimensional space, with color information for each point. This data is transferred over the network to create a holographic effect, allowing users to view objects from different angles. Adaptive streaming is used to transport holograms through point cloud scenes, adjusting the quality based on available bandwidth. Quality levels are dynamically changed based on user attention and bandwidth availability. Subjective studie

{'input': 'How much data will be necessary to send an hologram through our networks?',
 'output': 'The amount of data necessary to send a hologram through networks will depend on factors such as the complexity of the holographic scene, the quality levels desired, available bandwidth, and user attention. The data includes point cloud representations with color information for each point, and adaptive streaming adjusts the quality based on these factors to ensure an optimal viewing experience.'}

In [59]:
agent("How AI is impacting Medical Imaging Diagnostics?")



> Entering new AgentExecutor chain...
Thought: I should use the tool to find a relevant quote about AI's impact on Medical Imaging Diagnostics from the seminar transcript.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "AI impact on Medical Imaging Diagnostics"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: I don't have specific information on "AI impact on Medical Imaging Diagnostics" from the provided context.
Thought:I will try to find a relevant quote about AI's impact on Medical Imaging Diagnostics from the seminar transcript.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "AI impact on Medical Imaging Diagnostics"
}
``` 



> Entering new RetrievalQA chain...

> Finished chain.

Observation: I don't have specific information on "AI impact on Medical Imaging Diagnostics" from the provided context.
Thought:I will try to find a relevant quote about AI's impact on Medical Imaging Diagnostics from the seminar trans

{'input': 'How AI is impacting Medical Imaging Diagnostics?',
 'output': 'I don\'t have specific information on "AI impact on Medical Imaging Diagnostics" from the provided context.'}

In [60]:
text = " How AI is impacting Medical Imaging Diagnostics?"

querry = f"""Please gather more information about '''{text}''' and always send me a string"""

qa_stuff.run(querry)



> Entering new RetrievalQA chain...

> Finished chain.


"I don't know."

In [64]:
agent("How AI is impacting Medical Imaging Diagnostics?")



> Entering new AgentExecutor chain...
Thought: I should use the tool find_seminar_quote to gather information about how AI is impacting Medical Imaging Diagnostics.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "AI impacting Medical Imaging Diagnostics"
}
``` 



> Entering new RetrievalQA chain...

> Finished chain.

Observation: I don't have specific information about AI impacting Medical Imaging Diagnostics from the provided context.
Thought:I should try using the tool find_seminar_quote again with a slightly different input to get more relevant information.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "AI in Medical Imaging Diagnostics"
}
``` 



> Entering new RetrievalQA chain...

> Finished chain.

Observation: I don't have information specifically on "AI in Medical Imaging Diagnostics" from the provided context.
Thought:I will try a broader input to see if I can gather relevant information about AI's impact on Medical Imaging Diagnos

{'input': 'How AI is impacting Medical Imaging Diagnostics?',
 'output': 'The application of AI in healthcare involves developing systems that analyze sensor data to provide valuable insights for healthcare professionals.'}

In [65]:
agent("what is the importance in time and latancy for a network and how it relate to edge computing?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I should find a relevant quote from a seminar that discusses the importance of time and latency in a network and its relation to edge computing.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "importance of time and latency in network and edge computing"
}
``` 



> Entering new RetrievalQA chain...

> Finished chain.

Observation: Time and latency are crucial factors in network and edge computing. In network computing, the challenge lies in ensuring that data is delivered within specific time constraints, which is essential for real-time applications. This is where time-sensitive networking comes into play, providing technology that guarantees data delivery with precision across multiple systems.

In edge computing, the focus is on reducing latency by processing data closer to the source, such as on edge devices like buses with embedded nodes. By aggregating and processing data on the edge be

{'input': 'what is the importance in time and latancy for a network and how it relate to edge computing?',
 'output': 'The importance of time and latency in a network is crucial for ensuring data is delivered within specific time constraints, especially for real-time applications. In edge computing, reducing latency by processing data closer to the source improves response times and efficiency. This emphasizes the critical role of time and latency in network and edge computing environments.'}

In [66]:
agent("why do birds know how to fly?")



> Entering new AgentExecutor chain...
Thought: I should use the find_seminar_quote tool to see if there is any relevant information about why birds know how to fly in the seminar transcript.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "why do birds know how to fly?"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: I don't have specific information on why birds know how to fly.
Thought:I will try to find information on why birds know how to fly in the seminar transcript.
Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "why do birds know how to fly?"
}
``` 


> Entering new RetrievalQA chain...

> Finished chain.

Observation: I don't have information on "why do birds know how to fly?" as it is not directly related to the context provided.
Thought:I will try to rephrase the question to see if I can find relevant information in the seminar transcript.

Action:
```
{
  "action": "find_seminar_quote",
  "action_input": "h

{'input': 'why do birds know how to fly?',
 'output': "I don't have specific information on why birds know how to fly based on the seminar transcript."}